In [51]:
pd.set_option('display.max_columns', 500)

In [52]:
print(packages_import_statement)

import statsmodels.formula.api as smf
import geopandas as gpd


import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
import os

import re
import time 

from pprint import pprint




In [53]:
##target variable
diversion_ratechange = pd.read_csv("../modeling_data/Y__Diversion_RateChange.csv")
diversion_ratechange.head(2)

,communitydistrict_index,Change_Diversion_Rate_PostLockdown
0,1__01,0.004018
1,1__02,-0.013964


In [54]:
cb_demos = gpd.read_file("../data/cb_demodata_geojson.json")
cb_to_nta = pd.read_csv("../data/nta_to_cb_clean.csv",skiprows=4,header=1)[['Borough','Unnamed: 4','Neighborhood Tabulation Area (NTA)']].iloc[1:]
cb_to_nta.columns = ['Borough',"CD","NTA"]

borough_to_code_dict =  {"MN":1
    ,"BX":2
    ,"BK":3
    ,"QN":4
    ,"SI":5}
def make_cb_index_code(row):
    b_code = borough_to_code_dict[row["CD"][0:2]]
    cd_code = row['CD'].split(" ")[-1]
    if len(cd_code)==1:
        cd_code = f"0{cd_code}"
    return f"{b_code}__{cd_code}"
    
cb_to_nta['communitydistrict_index']=cb_to_nta.apply(make_cb_index_code,axis=1)
cb_to_nta.sample(5)

,Borough,CD,NTA,communitydistrict_index
204,Queens,QN Community District 9,QN60,4__09
134,Manhattan,MN Community Districts 5,MN15,1__05
157,Queens,QN Community District 3,QN27,4__03
6,Bronx,BX Community District 12,BX62,2__12
214,Queens,QN Community District 10,QN57,4__10


In [55]:
cb_to_nta[cb_to_nta['CD'].str.contains("&")]

,Borough,CD,NTA,communitydistrict_index


In [56]:
### Some NTAs overlap CDs, so we just use the same neighborhoods for each CD with overlap

In [57]:
# cb_to_nta.communitydistrict_index.value_counts(),len(cb_to_nta.communitydistrict_index.value_counts())

In [58]:
estimate_columns = [c for c in cb_demos.columns if c[-1]=="E"]
cb_demos=cb_demos[["NTACode"]+estimate_columns]
cb_demos=cb_demos.merge(right=cb_to_nta[['NTA',"communitydistrict_index"]],left_on='NTACode',right_on='NTA',how='left')
cb_demos

,NTACode,maleE,femE,popu5E,pop5t9E,pop10t14E,pop15t19E,pop20t24E,pop25t29E,pop30t34E,pop35t39E,pop40t44E,pop45t49E,pop50t54E,pop55t59E,pop60t64E,pop65t69E,pop70t74E,pop75t79E,pop80t84E,pop85plE,popu181E,pop65pl1E,mdageE,popu18_2E,popu18mE,popu18fE,pop65pl2E,pop65plmE,pop65plfE,pop_2E,hsp1E,nhspE,wtnhE,blnhE,aiannhE,asnnhE,nhpinhE,othnhE,rc2plnhE,hsp2E,hspmeE,hspprE,hspcubE,hspdomE,hspcamE,hspcrE,hspguatmE,hsphndE,hspnicE,hsppanE,hspsalvE,hspocamE,hspsamE,hspargE,hspbolE,hspchlE,hspcolE,hspecE,hspprgE,hspperuE,hspurgE,hspvnzE,hsposamE,hspothE,hspspnrdE,hspspnshE,hspspamE,hspallothE,asn1rcE,asneastE,asnchinotE,asnjpnE,asnkorE,asnmgolE,asnokinwE,asntwnE,asnsouthE,asnindE,asnbngE,asnbhutnE,asnnepalE,asnpakE,asnsrilE,asnseastE,asnburmE,asncmbE,asnflpE,asnhmngE,asnindnsnE,asnlaoE,asnmalsnE,asnthaiE,asnvtnE,asnoasnE,asn2plE,NTA,communitydistrict_index
0,BK88,52209,50285,14342,11460,10824,7966,7028,6817,6408,5707,5346,3753,3416,3570,4674,4274,1781,1671,1308,2149,42136,11183,24.7,42136,21864,20272,11183,4826,6357,102494,8776,93718,78598,1059,101,13253,13,112,582,8776,3174,1723,167,862,1035,200,481,69,49,41,195,0,1386,205,0,0,139,995,0,6,40,1,0,429,131,22,0,276,13293,9616,9556,6,43,0,0,11,3146,1517,684,0,0,945,0,383,11,14,112,0,39,0,0,15,192,13,135,BK88,3__12
1,QN51,23927,26304,2751,1884,2352,2005,2665,3745,3407,3052,3481,4067,3921,3833,3234,2842,2263,1819,1409,1501,8282,9834,44.7,8282,4489,3793,9834,4293,5541,50231,7912,42319,11913,831,61,28526,40,323,625,7912,436,1194,158,827,2145,0,170,48,6,27,1894,0,2659,182,0,31,1006,981,40,333,19,67,0,493,90,39,0,364,28553,24714,15675,31,8490,0,0,518,2153,1870,28,0,58,172,25,935,72,0,606,0,54,0,22,0,181,114,637,QN51,4__07
2,QN27,9570,10220,1430,1209,1211,1057,1365,1497,1680,1248,1384,1310,1434,1244,895,953,517,538,385,433,4528,2826,36.8,4528,2143,2385,2826,1085,1741,19790,12007,7783,1042,4270,0,2206,0,34,231,12007,2174,363,22,2942,728,0,99,90,143,55,293,48,5043,30,15,40,1076,3443,55,331,0,42,11,735,228,27,0,480,2217,505,489,11,5,0,0,0,1402,774,347,0,111,164,6,295,19,0,231,0,0,16,0,0,29,7,8,QN27,4__03
3,QN07,10228,11220,1345,1085,1306,1309,1465,1875,1405,1287,1248,1500,1750,1477,1402,1109,668,364,438,415,4519,2994,38.6,4519,2194,2325,2994,1122,1872,21448,2204,19244,515,13679,1,3174,0,1198,677,2204,38,552,55,559,165,0,46,28,0,41,50,0,679,9,0,0,289,284,0,48,0,49,0,156,0,0,0,156,3184,42,36,0,6,0,0,0,2676,1921,722,0,0,33,0,233,0,0,228,0,0,0,0,5,0,209,24,QN07,4__12
4,MN06,10976,12697,1505,1070,971,1173,2583,3000,2164,1595,1360,1572,1505,1158,1298,724,711,594,338,352,4250,2719,33.5,4250,2039,2211,2719,876,1843,23673,12425,11248,2632,6920,35,1289,0,29,343,12425,1398,2484,220,6878,270,0,19,32,40,29,150,0,783,60,0,24,155,513,0,15,0,16,0,392,59,12,0,321,1289,943,741,72,130,0,0,0,216,216,0,0,0,0,0,130,0,0,95,0,0,0,0,0,35,0,0,MN06,1__09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,BX99,346,485,108,151,22,31,40,153,118,53,13,10,5,32,41,13,15,16,5,5,308,54,27.1,308,104,204,54,30,24,831,296,535,88,324,101,3,0,0,19,296,17,142,0,73,35,0,35,0,0,0,0,0,11,0,0,0,0,5,0,6,0,0,0,18,17,0,0,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,3,0,0,0,0,0,0,0,0,BX99,2__03
233,BX99,346,485,108,151,22,31,40,153,118,53,13,10,5,32,41,13,15,16,5,5,308,54,27.1,308,104,204,54,30,24,831,296,535,88,324,101,3,0,0,19,296,17,142,0,73,35,0,35,0,0,0,0,0,11,0,0,0,0,5,0,6,0,0,0,18,17,0,0,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,3,0,0,0,0,0,0,0,0,BX99,2__04
234,BX99,346,485,108,151,22,31,40,153,118,53,13,10,5,32,41,13,15,16,5,5,308,54,27.1,308,104,204,54,30,24,831,296,535,88,324,101,3,0,0,19,296,17,142,0,73,35,0,35,0,0,0,0,0,11,0,0,0,0,5,0,6,0,0,0,18,17,0,0,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,3,0,0,0,0,0,0,0,

In [59]:
pop_data_by_cd = cb_demos.groupby(by='communitydistrict_index').sum()
pop_data_by_cd

,maleE,femE,popu5E,pop5t9E,pop10t14E,pop15t19E,pop20t24E,pop25t29E,pop30t34E,pop35t39E,pop40t44E,pop45t49E,pop50t54E,pop55t59E,pop60t64E,pop65t69E,pop70t74E,pop75t79E,pop80t84E,pop85plE,popu181E,pop65pl1E,mdageE,popu18_2E,popu18mE,popu18fE,pop65pl2E,pop65plmE,pop65plfE,pop_2E,hsp1E,nhspE,wtnhE,blnhE,aiannhE,asnnhE,nhpinhE,othnhE,rc2plnhE,hsp2E,hspmeE,hspprE,hspcubE,hspdomE,hspcamE,hspcrE,hspguatmE,hsphndE,hspnicE,hsppanE,hspsalvE,hspocamE,hspsamE,hspargE,hspbolE,hspchlE,hspcolE,hspecE,hspprgE,hspperuE,hspurgE,hspvnzE,hsposamE,hspothE,hspspnrdE,hspspnshE,hspspamE,hspallothE,asn1rcE,asneastE,asnchinotE,asnjpnE,asnkorE,asnmgolE,asnokinwE,asntwnE,asnsouthE,asnindE,asnbngE,asnbhutnE,asnnepalE,asnpakE,asnsrilE,asnseastE,asnburmE,asncmbE,asnflpE,asnhmngE,asnindnsnE,asnlaoE,asnmalsnE,asnthaiE,asnvtnE,asnoasnE,asn2plE
communitydistrict_index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1__01,76784,79737,8214,5366,3993,6566,10566,20094,20057,15501,11256,10247,8116,8276,8343,6855,4760,2989,2206,3116,19724,19926,156.8,19724,9925,9799,19926,8880,11046,156521,10469,146052,111981,4395,105,23956,54,703,4858,10469,1754,2622,794,696,287,0,54,68,26,65,74,0,2028,349,58,133,567,246,0,325,95,246,9,2288,1056,208,0,1024,24101,17207,13669,1476,1732,0,0,330,4812,4185,146,0,63,312,106,1419,42,33,683,0,51,15,50,135,410,216,447
1__02,76784,79737,8214,5366,3993,6566,10566,20094,20057,15501,11256,10247,8116,8276,8343,6855,4760,2989,2206,3116,19724,19926,156.8,19724,9925,9799,19926,8880,11046,156521,10469,146052,111981,4395,105,23956,54,703,4858,10469,1754,2622,794,696,287,0,54,68,26,65,74,0,2028,349,58,133,567,246,0,325,95,246,9,2288,1056,208,0,1024,24101,17207,13669,1476,1732,0,0,330,4812,4185,146,0,63,312,106,1419,42,33,683,0,51,15,50,135,410,216,447
1__03,75911,79084,4687,4435,5161,8433,12249,19598,15049,10936,9181,10029,8924,9463,9179,8001,6363,4412,4087,4808,17670,27671,117.3,17670,9450,8220,27671,11738,15933,154995,38899,116096,51603,12980,438,47412,32,270,3361,38899,1558,23154,337,6992,1392,40,76,436,35,276,498,31,3072,527,16,189,651,881,7,397,51,215,138,2394,568,232,50,1544,47811,43905,41124,1057,1643,20,0,61,2282,1749,393,0,0,53,87,1199,0,6,586,0,131,0,149,47,280,47,378
1__04,80900,73596,5172,3095,2679,4336,11259,21929,21039,13843,11943,10515,9123,9892,8919,7003,5306,3903,2325,2215,12663,20752,112.8,12663,5878,6785,20752,9640,11112,154496,23906,130590,91370,8077,193,26538,80,885,3447,23906,3000,7073,1175,2967,1247,87,174,228,208,37,513,0,5962,776,198,224,1445,2127,129,523,64,476,0,2482,626,530,0,1326,26692,18374,11723,1924,4065,0,0,662,5120,4372,157,0,42,442,107,2064,18,78,1273,0,61,0,11,311,312,512,622
1__05,80900,73596,5172,3095,2679,4336,11259,21929,21039,13843,11943,10515,9123,9892,8919,7003,5306,3903,2325,2215,12663,20752,112.8,12663,5878,6785,20752,9640,11112,154496,23906,130590,91370,8077,193,26538,80,885,3447,23906,3000,7073,1175,2967,1247,87,174,228,208,37,513,0,5962,776,198,224,1445,2127,129,523,64,476,0,2482,626,530,0,1326,26692,18374,11723,1924,4065,0,0,662,5120,4372,157,0,42,442,107,2064,18,78,1273,0,61,0,11,311,312,512,622
1__06,68839,78076,5276,3292,2948,4788,13078,22981,16382,11117,8598,8016,7867,8009,7788,8180,6796,4539,3443,3817,12982,26775,147.4,12982,6331,6651,26775,11229,15546,146915,10710,136205,102427,5004,300,24337,47,667,3423,10710,1467,3098,1043,600,512,53,80,27,103,169,80,0,2348,564,87,83,775,242,18,360,74,145,0,1642,878,188,0,576,24540,15833,9481,1837,3821,71,0,623,5743,4833,144,0,86,610,70,2002,0,33,1505,0,5,0,172,115,172,461,501
1__07,89253,107772,10824,8066,7616,5821,6853,16313,19256,14953,14492,13321,13549,12699,12940,13063,10882,5831,4052,6494,30218,40322,134.0,30218,14301,15917,40322,16656,23666,197025,28962,168063,133011,11477,95,17905,32,907,4636,28962,3235,9136,1095,8320,1009,31,329,260,71,208,72,38,3052,338,65,184,612,971,30,615,0,237,0,3115,1416,406,0,1293,17930,11538,7317,1349,2532,0,0,340,4209,3725,43,0,47,290,104,1353,0,10,940,0,26,6,0,153,218,284,546
1__08,

In [60]:
cb_econ = gpd.read_file("../data/cb_econdata_geojson.json")
estimate_columns = [c for c in cb_econ.columns if c[-1]=="E"]
cb_econ=cb_econ[["NTACode"]+estimate_columns]
cb_econ=cb_econ.merge(right=cb_to_nta[['NTA',"communitydistrict_index"]],left_on='NTACode',right_on='NTA',how='left')
cb_econ

,NTACode,lfE,cvlf1E,cvem16pl1E,cvlfuem1E,lfarmdfE,nlf1E,cvlf2E,cvlfuem2E,f16plE,f16pllfE,f16plcvlfE,f16plclfeE,ochu6E,ochu6plfE,och6t17E,oc6t17plfE,wrkr16plE,cw_drvalnE,cw_crpldE,cw_pbtrnsE,cw_wlkdE,cw_othE,cw_wrkdhmE,mntrvtmE,cvem16pl2E,mgbsciartE,srvcE,salesoffE,nrcnstmntE,prdtrnsmmE,cvem16pl3E,agffhmE,constctnE,mnfctrngE,whlsaleE,retailE,trwhutE,infoE,fireE,prfsmgawmE,edhlthcsaE,artenrafsE,othnotpaE,pubadminE,cvem16pl4E,prvwswrkrE,gvtwrkrE,slfemnincE,updfmwrkrE,hh2E,hhiu10E,hhi10t14E,hhi15t24E,hhi25t34E,hhi35t49E,hhi50t74E,hhi75t99E,hi100t149E,hi150t199E,hhi200plE,mdhhincE,mnhhincE,inc_sosecE,inc_rtrmtE,inc_spsecE,inc_cpbaE,inc_snapE,fam2E,famiu10E,fami10t14E,fami15t24E,fami25t34E,fami35t49E,fami50t74E,fami75t99E,fi100t149E,fi150t199E,fami200plE,mdfamincE,nfam2E,mdnfincE,percapincE,mdewrkE,mdemftwrkE,mdefftwrkE,cvnipop2E,hinsE,pvhinsE,pbhinsE,nhinsE,cvniu18_2E,u18nhinsE,cni1864_2E,cvlf18t64E,cvlfemE,emhinsE,empvhinsE,empbhinsE,emnhinsE,uemE,uemhinsE,uempvhinsE,uempbhinsE,uemnhinsE,nlf2E,nlfhinsE,nlfpvhinsE,nlfpbhinsE,nlfnhinsE,fampvuE,fambwpvE,poppvu1E,pbwpvE,pu18pvuE,pu18bwpvE,p65plpvuE,p65plbwpvE,poppvu2E,pvu50E,pv50t74E,pv75t99E,pv100t124E,pv125t149E,pv150t174E,pv175t184E,pv185t199E,pv200t299E,pv300t399E,pv400t499E,pv500plE,NTA,communitydistrict_index
0,BK88,36588,36575,34782,1793,13,27207,36575,1793,31738,15977,15977,15112,16576,8075,24846,13685,33725,6800,2781,11611,10104,824,1605,33.6,34782,12096,6240,10985,2577,2884,34782,56,2480,1619,2063,4247,1434,589,2560,3000,11007,3319,1975,433,34782,30203,2184,2336,59,25857,1887,2283,4157,3378,3128,3488,2404,2764,1258,1110,41018,64990.0,6601,2235,1760,1208,9071,20940,1075,1516,3368,2822,2814,2822,2066,2370,1063,1024,43694,4917,23244,16612.0,21781,41847,37935,101942,95811,33643,69709,6131,43559,614,47568,33293,31708,27711,13689,15170,3997,1585,1365,591,825,220,14275,13087,4514,9267,1188,20940,6444,101653,36835,41902,19786,10815,2277,101653,12509,12917,11409,7468,7133,5502,2358,2704,13865,9300,5537,10951,BK88,3__12
1,QN51,24047,24036,23024,1012,11,18765,24036,1012,22925,10957,10957,10577,3024,1657,5033,3043,22368,7197,2658,9295,2676,267,275,40.2,23024,7294,6179,5337,1763,2451,23024,56,1437,860,600,2684,1516,585,1877,2319,4450,3234,2644,762,23024,19468,2411,1133,12,17425,1428,802,2364,1786,1856,2980,1981,2293,928,1007,53758,73597.0,5412,2166,1238,752,1974,12207,647,386,1254,1330,1454,2205,1579,1771,758,823,61249,5218,28980,26619.0,31198,41451,37408,49528,41132,25771,18686,8396,8630,987,31661,22843,21941,17063,13847,3473,4878,902,703,376,355,199,8818,6716,3508,3537,2102,12207,1520,49484,7523,8227,1133,9237,1879,49484,3049,1235,3239,3896,2507,3305,1243,1473,8614,5623,4493,10807,QN51,4__07
2,QN27,9442,9442,8807,635,0,6230,9442,635,8038,4265,4265,3925,1533,1044,2664,1817,8069,2008,721,4542,433,237,128,44.3,8807,1786,2783,1665,1134,1439,8807,0,1015,421,108,885,781,165,421,902,1830,1476,594,209,8807,7265,908,634,0,5541,426,215,592,607,864,870,721,816,236,194,52231,67905.0,1630,919,407,229,971,3794,158,75,334,381,594,654,621,648,165,164,64521,1747,28058,20549.0,27967,36825,33864,19346,16402,8899,8956,2944,4684,229,12192,9003,8438,6714,4911,2114,1724,565,437,208,268,128,3189,2464,1116,1558,725,3794,547,19228,3415,4410,1066,2470,427,19228,1181,1032,1202,868,1543,1338,179,827,4065,2353,1804,2836,QN27,4__03
3,QN07,12104,12104,11208,896,0,5330,12104,896,9139,5871,5871,5503,1400,1126,2808,2293,11038,4862,289,5241,249,122,275,49.3,11208,3163,3216,2426,1023,1380,11208,0,682,186,163,1236,1399,265,624,1046,3601,892,600,514,11208,8893,1731,542,42,6298,303,239,543,393,567,1138,906,1031,706,472,74216,88956.0,1731,1132,359,327,996,4784,112,106,353,222,477,743,769,889,679,434,87321,1514,33425,28133.0,35594,49478,44132,21063,19208,12782,7765,1855,4789,86,13604,11308,10488,9406,7735,1803,1082,820,617,305,316,203,2296,1933,1051,987,363,4784,398,20895,2273,4351,727,2670,196,20895,1073,596,604,699,581,801,438,427,2838,3378,2907,6553,QN07,4__12
4,MN06,12024,12024,11293,731,0,7914,12024,731,10673,6061,6061,56

In [61]:
econ_data_by_cd = cb_econ.groupby(by='communitydistrict_index').sum()
econ_data_by_cd

,lfE,cvlf1E,cvem16pl1E,cvlfuem1E,lfarmdfE,nlf1E,cvlf2E,cvlfuem2E,f16plE,f16pllfE,f16plcvlfE,f16plclfeE,ochu6E,ochu6plfE,och6t17E,oc6t17plfE,wrkr16plE,cw_drvalnE,cw_crpldE,cw_pbtrnsE,cw_wlkdE,cw_othE,cw_wrkdhmE,mntrvtmE,cvem16pl2E,mgbsciartE,srvcE,salesoffE,nrcnstmntE,prdtrnsmmE,cvem16pl3E,agffhmE,constctnE,mnfctrngE,whlsaleE,retailE,trwhutE,infoE,fireE,prfsmgawmE,edhlthcsaE,artenrafsE,othnotpaE,pubadminE,cvem16pl4E,prvwswrkrE,gvtwrkrE,slfemnincE,updfmwrkrE,hh2E,hhiu10E,hhi10t14E,hhi15t24E,hhi25t34E,hhi35t49E,hhi50t74E,hhi75t99E,hi100t149E,hi150t199E,hhi200plE,mdhhincE,mnhhincE,inc_sosecE,inc_rtrmtE,inc_spsecE,inc_cpbaE,inc_snapE,fam2E,famiu10E,fami10t14E,fami15t24E,fami25t34E,fami35t49E,fami50t74E,fami75t99E,fi100t149E,fi150t199E,fami200plE,mdfamincE,nfam2E,mdnfincE,percapincE,mdewrkE,mdemftwrkE,mdefftwrkE,cvnipop2E,hinsE,pvhinsE,pbhinsE,nhinsE,cvniu18_2E,u18nhinsE,cni1864_2E,cvlf18t64E,cvlfemE,emhinsE,empvhinsE,empbhinsE,emnhinsE,uemE,uemhinsE,uempvhinsE,uempbhinsE,uemnhinsE,nlf2E,nlfhinsE,nlfpvhinsE,nlfpbhinsE,nlfnhinsE,fampvuE,fambwpvE,poppvu1E,pbwpvE,pu18pvuE,pu18bwpvE,p65plpvuE,p65plbwpvE,poppvu2E,pvu50E,pv50t74E,pv75t99E,pv100t124E,pv125t149E,pv150t174E,pv175t184E,pv185t199E,pv200t299E,pv300t399E,pv400t499E,pv500plE
communitydistrict_index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1__01,101888,101871,97888,3983,17,36395,101871,3983,70692,49678,49671,47413,9164,6056,10189,6482,96632,4404,1273,50426,25429,6932,8168,76.6,97888,71247,5606,18616,885,1534,97888,11,750,1950,2204,6910,1116,7529,22910,26612,15540,8072,2696,1588,97888,85281,4684,7856,67,77640,3470,1976,3358,2755,4030,6605,6340,12171,8739,28196,435327,694109.0,13839,6791,1391,612,2923,29236,507,399,725,627,869,1892,1865,3609,3111,15632,609116,48404,346204,354848.0,285346,314999,299958,155893,150315,134220,27558,5578,22384,519,113641,94831,91236,88250,85522,3383,2986,3595,3015,2253,883,580,18810,17526,14678,3363,1284,29236,1120,149020,11955,19473,583,19868,2382,149020,6509,2409,3037,2796,1888,2280,815,1038,8315,9055,8724,102154
1__02,101888,101871,97888,3983,17,36395,101871,3983,70692,49678,49671,47413,9164,6056,10189,6482,96632,4404,1273,50426,25429,6932,8168,76.6,97888,71247,5606,18616,885,1534,97888,11,750,1950,2204,6910,1116,7529,22910,26612,15540,8072,2696,1588,97888,85281,4684,7856,67,77640,3470,1976,3358,2755,4030,6605,6340,12171,8739,28196,435327,694109.0,13839,6791,1391,612,2923,29236,507,399,725,627,869,1892,1865,3609,3111,15632,609116,48404,346204,354848.0,285346,314999,299958,155893,150315,134220,27558,5578,22384,519,113641,94831,91236,88250,85522,3383,2986,3595,3015,2253,883,580,18810,17526,14678,3363,1284,29236,1120,149020,11955,19473,583,19868,2382,149020,6509,2409,3037,2796,1888,2280,815,1038,8315,9055,8724,102154
1__03,79625,79625,74614,5011,0,60062,79625,5011,71886,38294,38294,36170,5492,3428,10795,6135,73274,3665,1797,39746,18410,6102,3554,93.4,74614,39398,14828,14818,1555,4015,74614,105,1449,2517,1251,6141,2519,5117,8477,16141,15292,10602,3336,1667,74614,61490,6520,6121,483,71832,9411,7644,8673,6103,6699,8499,5740,8218,3845,7000,147887,253053.0,19876,7564,8923,3746,18426,29644,2470,2650,4323,3215,3923,3649,2242,2678,1378,3116,169431,42188,138148,127006.0,136780,205114,179220,153205,144142,83401,69352,9063,20131,291,105698,75847,71147,66463,55421,12501,4684,4700,3917,1257,2765,783,29851,26785,11574,16452,3066,29644,6916,147953,38677,17465,6612,27376,9012,147953,15955,8852,13870,9689,8450,6853,2628,3103,17664,11693,9985,39211
1__04,107924,107886,102855,5031,38,34881,107886,5031,67326,47203,47165,44714,5666,3885,6469,4478,101476,4393,970,49517,33280,6529,6787,78.3,102855,70713,9062,19422,1409,2249,102855,98,1496,3082,2199,6879,1652,8470,23020,25287,14924,10024,3503,2221,102855,89104,5684,8059,8,87912,6564,3860,5820,4565,5384,8410,7663,13055,8993,23598,330353,514331.0,15977,7250,3140,2224,7455,25440,747,616,1308,1037,1005,1850,2101,3159,2794,10823,504994,62472,268

In [62]:
columns_to_save_demo = ['popu18_2E',"pop65pl2E","wtnhE","blnhE","hsp1E","asnnhE","othnhE"]
columns_to_save_econ = ["cvem16pl1E","cvlfuem1E","hhiu10E"\
                        ,"hhi10t14E","hhi15t24E","hhi25t34E"\
                       ,"hhi35t49E","hhi50t74E","hhi75t99E"\
                       ,"hi100t149E","hi150t199E","hhi200plE"\
                       ,"inc_snapE","infoE",'fireE',"prfsmgawmE","edhlthcsaE","artenrafsE","othnotpaE","pubadminE"]

pop_data_by_cd[columns_to_save_demo]\
.rename({'popu18_2E':"pop_under_18","pop65pl2E":"pop_over_65"\
        ,"wtnhE":"white_alone","blnhE":"black_alone"\
         ,"hsp1E":"hispanic","asnnhE":"asian_alone","othnhE":"other_race_alone"},axis=1)\
.to_csv("../modeling_data/Xs_ACS_DemoData.csv")

econ_data_by_cd[columns_to_save_econ]\
.rename({"cvem16pl1E":"num_employed","cvlfuem1E":"num_unemployed","hhiu10E":"hh_inc<10K"\
    ,"hhi10t14E":"hh_inc10-14K","hhi15t24E":"hh_inc15-24K","hhi25t34E":"hh_inc25-34K"\
   ,"hhi35t49E":"hh_inc35-49K","hhi50t74E":"hh_inc50-74K","hhi75t99E":"hh_inc75-99K"\
   ,"hi100t149E":"hh_inc100-149K","hi150t199E":"hh_inc150-199K","hhi200plE":"hh_inc>200K"\
   ,"inc_snapE":"hh_with_snap_past12mos","infoE":"num_eployed_info_sector"
         ,'fireE':"num_eployed_fire_sector","prfsmgawmE":"num_eployed_sciences_waste_sector"\
          ,"edhlthcsaE":"num_eployed_educhealthsocial_sector"\
         ,"artenrafsE":"num_eployed_artsentertainmentfoood_sector"\
         ,"othnotpaE":"num_eployed_other_sector","pubadminE":"num_eployed_publicadmin_sector"},axis=1)\
.to_csv("../modeling_data/Xs_ACS_EconData.csv")